<a href="https://colab.research.google.com/github/aipatchouli/JAX_Demo/blob/main/CNN_JAX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/googledrive')

Mounted at /content/googledrive


In [ ]:
!pip install -Uq tensorflow-datasets jax optax flax dm-haiku rlax tensorflow
!pip show jax | grep "Version"
!cat /proc/cpuinfo | grep "model name"
!nvidia-smi

In [5]:
import jax
jax.devices()

[CpuDevice(id=0)]

In [ ]:
from jax import jit, grad, vmap
import jax.numpy as jnp
from jax import random
import numpy as np

key = random.PRNGKey(0)